<center><h1>Article Crawler</h1></center>
<center><h3>"Everything in the internet is legal !" - Abraham Lincoln</h3></center>

<br>Ussualy when you need dataset for article classification, either it is sentiment analysis or category-like classification, you will crawl articles from publisher site. In this notebook, we will give tutorial about crawling articles from a publisher site. <br><br>
First, we need import two important library : <br>
- BeatifulSoup : Saving HTML data into useful data structure. This useful for retrieve content and title from publisher site. Further explanation (https://www.crummy.com/software/BeautifulSoup/bs4/doc/)<br>
- Requests : Get HTML data from publisher site.

In this notebook, we choose [timorexpress](http://timorexpress.fajar.co.id/) as publisher site to crawl. We want to crawl [*rakyat-timor*](http://timorexpress.fajar.co.id/rubrik/rakyat-timor/page/1/) rubric :

![tribun-timur](img/timur.png)

As you can see, currently there are 44 pages of *rakyat-timor* rubric and we want to crawl all article in that section.


In [3]:
from bs4 import BeautifulSoup
import requests


Our main crawler application need two function :
- parse_list(url_publisher) : Retrieve 10 articles url from [this](http://timorexpress.fajar.co.id/rubrik/rakyat-timor/page/1/) page. Returned result from this function is list of articles url.
- parse_detail(url_article) : Extract title and content from an aritcle url. After we get list of url from *parse_list* we will iterate each url and extract the title and content, this is where parse_detail work.

Before you create parse_list function, you need to know where is url article store in HTML file. Here is the step:
1. Open [this](http://timorexpress.fajar.co.id/rubrik/rakyat-timor/page/1/) page
2. Right click in that page, choose inspect element.
3. Try highlight an article like this : <br>
![ss2](img/ss2.png) <br>
4. Look at the HTML, the article is in `<li class='infinite-post'>`. 
5. Usualy, the url is in `<a href="blabla">` part. In this case, the url in `<a href="http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/">` : <br>
![ss3](img/ss3.png) <br>
6. Those HTML adress is used to get article url.

Now look at our example below.

 First, we get HTML data from the page.

In [9]:
resp = requests.get("http://timorexpress.fajar.co.id/rubrik/rakyat-timor/page/1/")

If you print the text attribute from the requests return, you will get HTML data like example below. This HTML file will be converted to BeautifulSoup(BS) data type.

In [52]:
print(resp.text)

BS will convert HTML data into fetchable data type. <br>In example you wanna get the `<script type="text/javascript">` part, use this syntax : `bs.find('script', {'type':'text/javascript'})`

<b>(Note : use `bs.find_all(HTML_PART)` syntax if you want to retrieve list all those kind of HTML part. If you only need one data, use `bs.find(HTML_PART)`)</b>

In [16]:
bs = BeautifulSoup(resp.text, "html.parser")
print(bs.find('script', {'type':'text/javascript'}))

<script type="text/javascript">
_atrk_opts = { atrk_acct:"tww3n1aMp4107i", domain:"fajar.co.id",dynamic: true};
(function() { var as = document.createElement('script'); as.type = 'text/javascript'; as.async = true; as.src = "https://d31qbv1cthcecs.cloudfront.net/atrk.js"; var s = document.getElementsByTagName('script')[0];s.parentNode.insertBefore(as, s); })();
</script>


From example above, you can fetch every HTML part in the data. In this case, you wanna get url article that saved in : <br>
`<a href="http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/">` <br>

You would think 'Yeah, easy, just fetch all `<a href>` part in the HTML'. But, if you fetch all `<a href>` part this will happen: 

In [21]:
print(bs.find_all('a', href=True))

[<a class="wallpaper-link" href="http://fajar.co.id" target="_blank"></a>, <a href="https://www.facebook.com/TimorExpressNews/" target="_blank">
<i class="fa fa-facebook-square fa-2"></i>
</a>, <a href="#" target="_blank">
<i class="fa fa-twitter fa-2"></i>
</a>, <a href="#" target="_blank">
<i class="fa fa-pinterest fa-2"></i>
</a>, <a href="#" target="_blank">
<i class="fa fa-google-plus fa-2"></i>
</a>, <a href="#" target="_blank">
<i class="fa fa-youtube-play fa-2"></i>
</a>, <a href="http://timorexpress.fajar.co.id/feed/rss/" target="_blank">
<i class="fa fa-rss fa-2"></i>
</a>, <a href="http://fajar.co.id">FAJAR.CO.ID</a>, <a href="#">COMMUNITY</a>, <a href="http://warkopanime.fajar.co.id/">Warkop Anime</a>, <a href="http://answer.fajar.co.id/">Fajar Answer</a>, <a href="http://fajaroke.com/">Fajar Oke</a>, <a href="http://burungkicau.fajar.co.id/">Klub Burung Kicau</a>, <a href="http://jualbeli.fajar.co.id/">Fajar Jual Beli</a>, <a href="http://loker.fajar.co.id/">Info Loker</a>

You will get all HTML part that contain `<a href>`. It like all url in those page.
<br>
So, we need to be specific to choose what kind of `<a href>` do we need. That's why we need to get into `article` part to get the right url. The true `article` is in `<li class='infinite-post'>`. So if we doing this :

In [22]:
bs = bs.find_all('li',{'class':'infinite-post'})
bs[0]

<li class="infinite-post">
<a href="http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/" rel="bookmark" title="Danrem 161/WS Kunjungi Pulau Kera">
<div class="archive-list-out">
<div class="archive-list-img left relative">
<img alt="" class="reg-img wp-post-image" height="270" sizes="(max-width: 450px) 100vw, 450px" src="http://timorexpress.fajar.co.id/wp-content/uploads/2017/09/ular-ijo-kpg-450x270.jpg" srcset="http://timorexpress.fajar.co.id/wp-content/uploads/2017/09/ular-ijo-kpg-450x270.jpg 450w, http://timorexpress.fajar.co.id/wp-content/uploads/2017/09/ular-ijo-kpg-300x180.jpg 300w" width="450"/> <img alt="" class="mob-img wp-post-image" height="80" sizes="(max-width: 80px) 100vw, 80px" src="http://timorexpress.fajar.co.id/wp-content/uploads/2017/09/ular-ijo-kpg-80x80.jpg" srcset="http://timorexpress.fajar.co.id/wp-content/uploads/2017/09/ular-ijo-kpg-80x80.jpg 80w, http://timorexpress.fajar.co.id/wp-content/uploads/2017/09/ular-ijo-kpg-150x150.jpg 150w" 

We are inside the article! <br>
We only need to retrieve the URL. Use this code:

In [24]:
bs[0].find('a', href=True)['href']

'http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/'

Wrap all the code into parse_list. Because we need all `article` part, we will iterate to `article` part get the url.

In [26]:
def parse_list(url_publisher):
    resp = requests.get(url_publisher)
    bs = BeautifulSoup(resp.text, "html.parser")
    list_of_url = []
    bs = bs.find_all('li',{'class':'infinite-post'})
    for a in bs:
        list_of_url.append(a.find('a',href=True)['href'])
    return list_of_url

Lets try our function.

In [27]:
print(parse_list("http://timorexpress.fajar.co.id/rubrik/rakyat-timor/page/1/"))

['http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/', 'http://timorexpress.fajar.co.id/2017/09/12/jonathan-banunaek-daftar-ke-gerindra/', 'http://timorexpress.fajar.co.id/2017/09/11/mantan-kepala-bppd-ttu-diperiksa/', 'http://timorexpress.fajar.co.id/2017/09/09/pengajuan-hak-angket-kandas/', 'http://timorexpress.fajar.co.id/2017/09/08/sman-1-amanuban-timur-bangun-lapangan-terintegrasi/', 'http://timorexpress.fajar.co.id/2017/09/08/maria-akhiri-hidup-dengan-foltus/', 'http://timorexpress.fajar.co.id/2017/09/07/johannis-ottemoesoe-pimpin-knpi-kabupaten-kupang/', 'http://timorexpress.fajar.co.id/2017/09/06/jemi-haekase-daftar-ke-pkb/', 'http://timorexpress.fajar.co.id/2017/09/05/ceburkan-diri-ke-embung-meri-tewas/']


Now, you can get all article url in page 1.

For parse_detail, you will doing the same step as parse_list.

1. Go to the page of an article to know where is the content. In this case we choose [this](http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/).
2. Inspect the HTML, highlight the text to get the content HTML part like this : <br>
![ss4](img/ss4.png)
<br>
3. If you notice, all content is saved in all `<p>` type. One paraghraph is saved in one `<p>`.

Now let do the code:


In [30]:
r = requests.get("http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/")
bs = BeautifulSoup(r.text, "html.parser")
print(bs.find_all('p'))

[<p>POSE BERSAMA. Danrem 161/Wira Sakti, Brigjen TNI Teguh Muji Angkasa pose bersama warga pulau Kera usai baksos, Sabtu (9/9)</p>, <p style="padding-left: 30px;"><strong>Bagi Sembako untuk Warga Pulau Kera dan Uiasa</strong></p>, <p><strong>OELAMASI, TIMEX</strong> – Menjelang HUT ke-72 TNI, Korem 161/Wira Sakti menggelar bakti sosial (Baksos) pembagian sembako kepada masyarakat pulau Kera dan masyarakat Desa Uiasa Kecamatan Semau, Sabtu (9/9).</p>, <p>Kegiatan baksos dipimpin Danrem 161/Wira Sakti (WS), Brigjen TNI Teguh Muji Angkasa, didampingi Ketua Persit KCK Koorcab Rem 161 PD IX/Udayana, Erny Teguh Muji Angkasa, Kasrem, para Kasi dan Dandim 1604/Kupang.</p>, <p>Kehadiran Danrem beserta rombongan di dua lokasi tersebut mendapat sambutan hangat tokoh masyarakat dan warga setempat.<br/>
Di pulau Kera, masyarakat yang sudah berkumpul di halaman Mushola Dahrul Bahar, menanti kesempatan bertatap muka dengan Danrem.<br/>
Salah satu Ketua RW di pulau Kera, Hamdan Saba menyampaikan ucapa

We are so lucky! No other `<p>` elsewhere in this page. <br><br>
<b> Note :  we can get the clean text by call `get_text()` in your bs4 attribute. Example : `bs.find('p').get_text()` <br></b>
Doing some processing, we get :

In [31]:
content = ''
for paraghrap in bs.find_all('p'):
    content += paraghrap.get_text()
print(content)

POSE BERSAMA. Danrem 161/Wira Sakti, Brigjen TNI Teguh Muji Angkasa pose bersama warga pulau Kera usai baksos, Sabtu (9/9)Bagi Sembako untuk Warga Pulau Kera dan UiasaOELAMASI, TIMEX – Menjelang HUT ke-72 TNI, Korem 161/Wira Sakti menggelar bakti sosial (Baksos) pembagian sembako kepada masyarakat pulau Kera dan masyarakat Desa Uiasa Kecamatan Semau, Sabtu (9/9).Kegiatan baksos dipimpin Danrem 161/Wira Sakti (WS), Brigjen TNI Teguh Muji Angkasa, didampingi Ketua Persit KCK Koorcab Rem 161 PD IX/Udayana, Erny Teguh Muji Angkasa, Kasrem, para Kasi dan Dandim 1604/Kupang.Kehadiran Danrem beserta rombongan di dua lokasi tersebut mendapat sambutan hangat tokoh masyarakat dan warga setempat.
Di pulau Kera, masyarakat yang sudah berkumpul di halaman Mushola Dahrul Bahar, menanti kesempatan bertatap muka dengan Danrem.
Salah satu Ketua RW di pulau Kera, Hamdan Saba menyampaikan ucapan terima kasih atas kunjungan Danrem dan rombongan dalam rangka baksos serta mengutarakan beberapa hal tentang k

Yes, we got all content. But, if you notice the last 4 paraghraph is contain dirty content like "(mg22/ays)FAJAR.CO.ID |" . We don't need it. So we doing this to get clean content:

In [34]:
content = ''
paraghraps = bs.find_all('p')
for paraghrap in paraghraps[:-4] :
    content += paraghrap.get_text()
print(content)

POSE BERSAMA. Danrem 161/Wira Sakti, Brigjen TNI Teguh Muji Angkasa pose bersama warga pulau Kera usai baksos, Sabtu (9/9)Bagi Sembako untuk Warga Pulau Kera dan UiasaOELAMASI, TIMEX – Menjelang HUT ke-72 TNI, Korem 161/Wira Sakti menggelar bakti sosial (Baksos) pembagian sembako kepada masyarakat pulau Kera dan masyarakat Desa Uiasa Kecamatan Semau, Sabtu (9/9).Kegiatan baksos dipimpin Danrem 161/Wira Sakti (WS), Brigjen TNI Teguh Muji Angkasa, didampingi Ketua Persit KCK Koorcab Rem 161 PD IX/Udayana, Erny Teguh Muji Angkasa, Kasrem, para Kasi dan Dandim 1604/Kupang.Kehadiran Danrem beserta rombongan di dua lokasi tersebut mendapat sambutan hangat tokoh masyarakat dan warga setempat.
Di pulau Kera, masyarakat yang sudah berkumpul di halaman Mushola Dahrul Bahar, menanti kesempatan bertatap muka dengan Danrem.
Salah satu Ketua RW di pulau Kera, Hamdan Saba menyampaikan ucapan terima kasih atas kunjungan Danrem dan rombongan dalam rangka baksos serta mengutarakan beberapa hal tentang k

To get the title, simply doing this:

In [35]:
bs.title.string

'Danrem 161/WS Kunjungi Pulau Kera – TIMOREXPRESS.COM'

To remove the name of website in the title we can easly replace the string with empty string

In [47]:
bs.title.string.replace(" – TIMOREXPRESS.COM", "")

'Danrem 161/WS Kunjungi Pulau Kera'

Now, wrap all the code.

In [48]:
def parse_detail(url_article):
    r = requests.get(url_article)
    bs = BeautifulSoup(r.text, "html.parser")
    title = bs.title.string.replace(" – TIMOREXPRESS.COM", "")
    paraghraps = bs.find_all('p')
    content = ''
    for paraghrap in paraghraps[:-4]:
        content += paraghrap.get_text()
    return (title, content, url_article)

Let's try.

In [49]:
parse_detail("http://timorexpress.fajar.co.id/2017/09/13/danrem-161ws-kunjungi-pulau-kera/")

('Danrem 161/WS Kunjungi Pulau Kera',
 'POSE BERSAMA. Danrem 161/Wira Sakti, Brigjen TNI Teguh Muji Angkasa pose bersama warga pulau Kera usai baksos, Sabtu (9/9)Bagi Sembako untuk Warga Pulau Kera dan UiasaOELAMASI, TIMEX – Menjelang HUT ke-72 TNI, Korem 161/Wira Sakti menggelar bakti sosial (Baksos) pembagian sembako kepada masyarakat pulau Kera dan masyarakat Desa Uiasa Kecamatan Semau, Sabtu (9/9).Kegiatan baksos dipimpin Danrem 161/Wira Sakti (WS), Brigjen TNI Teguh Muji Angkasa, didampingi Ketua Persit KCK Koorcab Rem 161 PD IX/Udayana, Erny Teguh Muji Angkasa, Kasrem, para Kasi dan Dandim 1604/Kupang.Kehadiran Danrem beserta rombongan di dua lokasi tersebut mendapat sambutan hangat tokoh masyarakat dan warga setempat.\nDi pulau Kera, masyarakat yang sudah berkumpul di halaman Mushola Dahrul Bahar, menanti kesempatan bertatap muka dengan Danrem.\nSalah satu Ketua RW di pulau Kera, Hamdan Saba menyampaikan ucapan terima kasih atas kunjungan Danrem dan rombongan dalam rangka baksos

You can make the main code. In example you want get all first 3 page of articles in rakyat-timor, surosa and flores-raya rubric.

In [50]:
list_url_parents = [
        'http://timorexpress.fajar.co.id/rubrik/rakyat-timor/page/',
        'http://timorexpress.fajar.co.id/rubrik/surosa/page/'
        'http://timorexpress.fajar.co.id/rubrik/flores-raya/page/'
    ]
articles = list()
for url_parents in list_url_parents:
    for page in range(0, 3):
        url_parent = url_parents+str(page)
        try:
            list_ = parse_list(url_parent)
        except:
            continue
        for url in list_:
            articles.append(parse_detail(url))

In [51]:
print("Total Article : ",  len(articles))
print("Example article : ", articles[0][1])

Total Article :  56
Example article :  POSE BERSAMA. Danrem 161/Wira Sakti, Brigjen TNI Teguh Muji Angkasa pose bersama warga pulau Kera usai baksos, Sabtu (9/9)Bagi Sembako untuk Warga Pulau Kera dan UiasaOELAMASI, TIMEX – Menjelang HUT ke-72 TNI, Korem 161/Wira Sakti menggelar bakti sosial (Baksos) pembagian sembako kepada masyarakat pulau Kera dan masyarakat Desa Uiasa Kecamatan Semau, Sabtu (9/9).Kegiatan baksos dipimpin Danrem 161/Wira Sakti (WS), Brigjen TNI Teguh Muji Angkasa, didampingi Ketua Persit KCK Koorcab Rem 161 PD IX/Udayana, Erny Teguh Muji Angkasa, Kasrem, para Kasi dan Dandim 1604/Kupang.Kehadiran Danrem beserta rombongan di dua lokasi tersebut mendapat sambutan hangat tokoh masyarakat dan warga setempat.
Di pulau Kera, masyarakat yang sudah berkumpul di halaman Mushola Dahrul Bahar, menanti kesempatan bertatap muka dengan Danrem.
Salah satu Ketua RW di pulau Kera, Hamdan Saba menyampaikan ucapan terima kasih atas kunjungan Danrem dan rombongan dalam rangka baksos se

Now, you ready to crawl a publisher site. The hardest part is always finding the right HTML part and make sure it not contain any other part. Also make sure your content is clean from any HTML, javascript, or unimportant content like example above. 

Disclaimer : We are not responsible if you get caught by anyone by crawl a site. (Be Safe)